# Computation Intelligence for Optimization | Sports League Optimization

`Group AM`
- Eduardo Mendes, 20240850
- Helena Duarte, 20240530
- João Freire, 20240528
- Mariana Sousa, 20240516

<div class="alert alert-block alert-info">

# Table of Contents
    
[1. Import Libraries](#1)<br>

[2. Load data](#2)<br>

<a class="anchor" id="1">

# 1. Import Libraries
    
</a>

In [1]:
import os
import pandas as pd

from copy import deepcopy
from random import random, sample, choice, randint
import copy

<a class="anchor" id="2">

# 2. Load data
    
</a>

In [2]:
data_dir= os.path.join(os.getcwd(), 'players(in).csv')

df = pd.read_csv(data_dir, index_col=0)
df.head()

,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


In [3]:
players = df.to_dict(orient="records")
players

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

<a class="anchor" id="3">

# 3. Problem Definiton
    
</a>

In a fantasy sports league, the objective is to assign players to teams in a way that ensures
a balanced distribution of talent while staying within salary caps.

1) Each player is defined by the following attributes:
* Skill rating: Represents the player's ability.
* Cost: The player's salary.
* Position (One of four roles) : Goalkeeper (GK), Defender (DEF), Midfielder (MID), or Forward (FWD).

A solution is a complete league configuration, specifying the team assignment for each player. These are the constraints that must be verified in every solution of the search space (no object is considered a solution if it doesn’t comply with these):
* Each team must consist of: 1 Goalkeeper, 2 Defenders, 2 Midfielders and 2
Forwards.
* Each player is assigned to exactly one team.

*Impossible Configurations*: Teams that do not follow this exact structure (e.g., a team with 2 goalkeepers, or a team where the same defender is assigned twice) are not part of the search space and are not considered solutions. It is forbidden to generate such an arrangement during evolution.

Besides that, each team should not exceed a 750€ million total budget. If it does, it is not a valid solution and the fitness value should reflect that.

The `objective` is to create a balanced league that complies with the constraints. 
A balanced league a is a league where the average skill rating of the players is roughly the same among the teams. 
This can be measured by the standard deviation of the average skill rating of the teams.

You can find a dataset of players with their names, position, skill rating and salary (in million €).
These players should be distributed across 5 teams of 7 players each.

In [4]:
def average_skill_rating(team):
    count = 0
    total_skill = 0

    for player in team:
        total_skill += player["Skill"]
        count += 1

    average = total_skill/count
    return average
def has_repeated_players(solution):

    players = []
    for team in solution: #5 teams
        for player in team: #7 players for each team
            if player['Name'] in players:
                return True
            players.append(player['Name'])
    
    return False

# 4. Fitness Function

<hr>

### THE ONE

In [100]:
import random
import copy
    
class League():
    def __init__(self, name, players):
        self.name = name
        self.players = players
        self.gk = []
        self.defenders = []
        self.midfielders = []
        self.forwards = []
        self.total_skill = 0
        self.average_skill = 0
        self.total_cost = 0

        for player in players:
            self.add_player(player)

    @staticmethod
    def random_teams(player_pool, num_teams=5, max_budget=750):
        positions = {
            'GK': [p for p in player_pool if p['Position'] == 'GK'],
            'DEF': [p for p in player_pool if p['Position'] == 'DEF'],
            'MID': [p for p in player_pool if p['Position'] == 'MID'],
            'FWD': [p for p in player_pool if p['Position'] == 'FWD']
        }
        
        used_players = set()
        teams = []

        for i in range(num_teams):
            number_tries = 0
            max_tries = 1000
            team = None

            while number_tries < max_tries:
                number_tries += 1
                try:
                    gk = random.choice([p for p in positions['GK'] if p['Name'] not in used_players])
                    def_players = random.sample([p for p in positions['DEF'] if p['Name'] not in used_players], 2)
                    mid_players = random.sample([p for p in positions['MID'] if p['Name'] not in used_players], 2)
                    fwd_players = random.sample([p for p in positions['FWD'] if p['Name'] not in used_players], 2)

                    team = [gk] + def_players + mid_players + fwd_players 
                    total_cost = sum(p["Salary (€M)"] for p in team)
                    total_skill = sum(p["Skill"] for p in team)
                    
                    if total_cost <= max_budget:
                         break

                except ValueError:
                    break

            if team is None:
                raise ValueError(f"Could not form a valid team {i+1} within budget after {max_tries} tries")

            used_players.update([p['Name'] for p in team])
            for p in team:
                positions[p['Position']].remove(p)
            teams.append((f"Team {i+1}", team))
        
        return teams

    @staticmethod
    def initialize_population(player_pool, num_leagues=3, num_teams=5, max_budget=750):
        all_solutions = []
        for league_num in range(num_leagues):
            # Generate ALL teams for the league at once
            league_teams = League.random_teams(
                copy.deepcopy(player_pool), 
                num_teams=num_teams,  # Generate 5 teams in one call
                max_budget=max_budget
        )
        
            print(f"League {league_num + 1}:")
            for team_idx, (team_name, team) in enumerate(league_teams, 1):
                total_cost = sum(p["Salary (€M)"] for p in team)
                avg_skill = sum(p["Skill"] for p in team) / len(team)
                print(f"  Team {team_idx}: {len(team)} players, Total Cost: {total_cost}M, Average Skill: {avg_skill:.2f}")
        
            all_solutions.append(league_teams)
    
        return all_solutions
        

    def valid_budget(self, budget):
        return self.calculate_total_cost() <= budget
    
    def calculate_total_skill(self):
        self.total_skill = sum(player["Skill"] for player in self.players)
        return self.total_skill
    
    def calculate_total_cost(self):
        self.total_cost = sum(player["Salary (€M)"] for player in self.players)
        return self.total_cost
    
    def calculate_avg_skill(self):
        self.average_skill = self.calculate_total_skill() / len(self.players) if self.players else 0
        return self.average_skill
    
    def fitness(self, budget):
        if not self.valid_budget(budget):
            return -100000
        else:
            cost_penalty = max(0, self.calculate_total_cost() - 100) * 5
            return self.calculate_total_skill() - cost_penalty
    
    # Magic methods
    def __str__(self):
        return f"Team: {self.name}, Players: {len(self.players)}, Average Skill: {self.average_skill}, Total Cost: {self.total_cost}"
    def __repr__(self):
        return f"Team({self.name}, {self.players})"
    def __len__(self):
        return len(self.players)
    def __getitem__(self, index):
        return self.players[index]
    def __setitem__(self, index, value):
        self.players[index] = value
    def __delitem__(self, index):
        del self.players[index]
    def __contains__(self, item):
        return item in self.players
    def __iter__(self):
        return iter(self.players)

    def add_player(self, player):
        pos = player['Position']
        if pos == 'GK':
            self.gk.append(player)
        elif pos == 'DEF':
            self.defenders.append(player)
        elif pos == 'MID':
            self.midfielders.append(player)
        elif pos == 'FWD':
            self.forwards.append(player)
        #self.players.append(player)

In [101]:
solution_leagues = League.initialize_population(players, num_leagues = 4, num_teams=5, max_budget=750)
#solution_leagues = population

League 1:
  Team 1: 7 players, Total Cost: 690M, Average Skill: 86.43
  Team 2: 7 players, Total Cost: 642M, Average Skill: 85.43
  Team 3: 7 players, Total Cost: 725M, Average Skill: 87.57
  Team 4: 7 players, Total Cost: 657M, Average Skill: 85.71
  Team 5: 7 players, Total Cost: 710M, Average Skill: 86.86
League 2:
  Team 1: 7 players, Total Cost: 675M, Average Skill: 86.43
  Team 2: 7 players, Total Cost: 640M, Average Skill: 85.00
  Team 3: 7 players, Total Cost: 657M, Average Skill: 85.14
  Team 4: 7 players, Total Cost: 702M, Average Skill: 86.86
  Team 5: 7 players, Total Cost: 750M, Average Skill: 88.57
League 3:
  Team 1: 7 players, Total Cost: 750M, Average Skill: 88.14
  Team 2: 7 players, Total Cost: 722M, Average Skill: 87.43
  Team 3: 7 players, Total Cost: 642M, Average Skill: 85.29
  Team 4: 7 players, Total Cost: 735M, Average Skill: 88.43
  Team 5: 7 players, Total Cost: 575M, Average Skill: 82.71
League 4:
  Team 1: 7 players, Total Cost: 720M, Average Skill: 87.57


In [102]:
for league_num, league in enumerate(solution_leagues, 1):
    print(f"League {league_num}:")
    for team_name, team_members in league:
        print(f"  {team_name}:")
        for player in team_members:
            print(f"    {player['Name']} ({player['Position']}, Skill: {player['Skill']}, Salary: {player['Salary (€M)']}M)")


League 1:
  Team 1:
    Ryan Mitchell (GK, Skill: 83, Salary: 85M)
    Mason Reed (DEF, Skill: 82, Salary: 75M)
    Logan Brooks (DEF, Skill: 86, Salary: 95M)
    Gavin Richardson (MID, Skill: 87, Salary: 95M)
    Dominic Bell (MID, Skill: 86, Salary: 95M)
    Colton Gray (FWD, Skill: 91, Salary: 125M)
    Xavier Bryant (FWD, Skill: 90, Salary: 120M)
  Team 2:
    Jordan Smith (GK, Skill: 88, Salary: 100M)
    Jaxon Griffin (DEF, Skill: 79, Salary: 65M)
    Maxwell Flores (DEF, Skill: 81, Salary: 72M)
    Connor Hayes (MID, Skill: 89, Salary: 105M)
    Nathan Wright (MID, Skill: 92, Salary: 120M)
    Landon Powell (FWD, Skill: 89, Salary: 110M)
    Tyler Jenkins (FWD, Skill: 80, Salary: 70M)
  Team 3:
    Alex Carter (GK, Skill: 85, Salary: 90M)
    Owen Parker (DEF, Skill: 88, Salary: 100M)
    Brayden Hughes (DEF, Skill: 87, Salary: 100M)
    Hunter Cooper (MID, Skill: 83, Salary: 85M)
    Ashton Phillips (MID, Skill: 90, Salary: 110M)
    Adrian Collins (FWD, Skill: 85, Salary: 90M)

In [ ]:
def fitness_solution(league_teams, budget=750):
    avg_skills = []

    for name, players in league_teams:
        team = League(name, players)
        if not team.valid_budget(budget):
            return -100000  

        avg_skills.append(team.calculate_avg_skill())

    std_dev = np.std(avg_skills)
    print(f"Average Skill :{std_dev}")
    return -std_dev 


In [106]:

solution_leagues = League.initialize_population(players, num_leagues = 4, num_teams=5, max_budget=750)

for i, league in enumerate(solution_leagues):
    score = fitness_solution(league)
    print(f"Liga {i+1} → Fitness: {score:.4f}")


League 1:
  Team 1: 7 players, Total Cost: 682M, Average Skill: 86.14
  Team 2: 7 players, Total Cost: 715M, Average Skill: 86.86
  Team 3: 7 players, Total Cost: 675M, Average Skill: 86.29
  Team 4: 7 players, Total Cost: 675M, Average Skill: 86.14
  Team 5: 7 players, Total Cost: 677M, Average Skill: 86.57
League 2:
  Team 1: 7 players, Total Cost: 675M, Average Skill: 85.71
  Team 2: 7 players, Total Cost: 602M, Average Skill: 84.14
  Team 3: 7 players, Total Cost: 725M, Average Skill: 87.86
  Team 4: 7 players, Total Cost: 740M, Average Skill: 87.71
  Team 5: 7 players, Total Cost: 682M, Average Skill: 86.57
League 3:
  Team 1: 7 players, Total Cost: 690M, Average Skill: 87.14
  Team 2: 7 players, Total Cost: 632M, Average Skill: 84.29
  Team 3: 7 players, Total Cost: 655M, Average Skill: 85.29
  Team 4: 7 players, Total Cost: 712M, Average Skill: 87.43
  Team 5: 7 players, Total Cost: 735M, Average Skill: 87.86
League 4:
  Team 1: 7 players, Total Cost: 700M, Average Skill: 86.00


<hr>